## Automated the process of applying EFA and LDA without Whitening to each subject 


In [1]:
from utility_func_final import * 

train_df = pickle.load(open("C:/Users/Admin/BCI_data/epoched_train.pkl", "rb"))
train_df = train_df.drop(['EOG:ch01','EOG:ch02','EOG:ch03'] , axis=1)

subject_data = {}

for i in range(1, 10):
    subject_key = f'subject_{i}'
    patient_ids = [f'B0{i}01T', f'B0{i}02T', f'B0{i}03T']
    
    subject_data[subject_key] = train_df[train_df['patient_id'].isin(patient_ids)].reset_index(drop = True)
    
# Create an empty dictionary to store accuracies
accuracy_per_subject_without_whitening = {}

# Iterate over each subject's data
for i in range(1, 10):
    train_df = subject_data[f'subject_{i}']

    # Split data into left and right-hand labels
    left_df = train_df[train_df['event_type'] == 0].reset_index(drop=True)
    right_df = train_df[train_df['event_type'] == 1].reset_index(drop=True)

    # Obtain mean sub images and eigen faces for left hand
    mean_sub_images_left = get_mean_sub_images(left_df)
    eigen_face_left = get_eigen_faces(mean_sub_images_left)
    left_coeff = get_coeff(mean_sub_images_left, eigen_face_left, label=0, trials=len(mean_sub_images_left[0]))

    # Obtain mean sub images and eigen faces for right hand
    mean_sub_images_right = get_mean_sub_images(right_df)
    eigen_face_right = get_eigen_faces(mean_sub_images_right)
    right_coeff = get_coeff(mean_sub_images_right, eigen_face_right, label=1, trials=len(mean_sub_images_right[0]))

    # Combine left and right coefficients to create feature DataFrame
    final_train_features_df = get_feature_df(left_coeff, right_coeff)

    # Evaluate accuracy
    accuracy, confusion_matrix_values = accuracy_through_lda(final_train_features_df)

    # Store accuracy in dictionary
    accuracy_per_subject_without_whitening[f"Subject_{i}"] =  round(accuracy * 100, 2)


## Automated the process of applying EFA and LDA with Whitening to each subject

In [2]:
from utility_func_final import *
accuracy_per_subject_with_whitening = {}

train_df = pickle.load(open("C:/Users/Admin/BCI_data/epoched_train.pkl", "rb"))
# train_df = pickle.load(open("C:/Users/Admin/BCI_data/epoched_train.pkl", "rb"))

for k in range(1, 10):
    subject_id_list = [f'B{k:02d}{j:02d}T' for j in range(1, 4)]
    subject_data = train_df[train_df['patient_id'].isin(subject_id_list)]
    
    subject_1 = subject_data

    rows = []
    for num in range(subject_1.shape[0]):
        c1 = subject_1.iloc[num,3]
        c2 = subject_1.iloc[num,4]
        c3 = subject_1.iloc[num,5]
        row= np.hstack([c1,c2,c3])
        rows.append(row)
    X = pd.DataFrame(rows)
    X = X.T
    
    event = subject_1['event_type']
    event=event.reset_index(drop = True)

    whitened_data = get_whiten_data(X)
    whitened_data['event_type'] = event 

    left_df = whitened_data[whitened_data['event_type'] == 0].reset_index(drop=True)
    right_df = whitened_data[whitened_data['event_type'] == 1].reset_index(drop=True)

    mean_sub_images_left = get_mean_sub_images(left_df, whiten_df=True)
    mean_sub_images_right = get_mean_sub_images(right_df, whiten_df=True)

    eigen_face_left = get_eigen_faces(mean_sub_images_left)
    eigen_face_right = get_eigen_faces(mean_sub_images_right)

    left_coeff = get_coeff(mean_sub_images_left, eigen_face_left, label=0, trials=len(mean_sub_images_left[0]))
    right_coeff = get_coeff(mean_sub_images_right, eigen_face_right, label=1, trials=len(mean_sub_images_left[0]))

    final_train_features_df = get_feature_df(left_coeff, right_coeff)
    
    accuracy, confusion_matrix_values = accuracy_through_lda(final_train_features_df)

    accuracy_per_subject_with_whitening[f"Subject_{k}"] = round(accuracy * 100, 2)



In [3]:
accuracy_df_without_whitening = pd.DataFrame(list(accuracy_per_subject_without_whitening.items()),
                                             columns=['Subject', 'Accuracy'])
accuracy_df_with_whitening = pd.DataFrame(list(accuracy_per_subject_with_whitening.items()), 
                                          columns=['Subject', 'Accuracy'])

accuracy_df_per_subject = pd.merge(accuracy_df_without_whitening, accuracy_df_with_whitening, on='Subject', how='inner')

accuracy_df_per_subject.loc[10] = {'Subject': 'Mean', 
                                    'Accuracy_x': accuracy_df_without_whitening['Accuracy'].mean(), 
                                   'Accuracy_y' : accuracy_df_with_whitening['Accuracy'].mean()}
accuracy_df_per_subject.loc[11] = {'Subject': 'Variance', 
                                    'Accuracy_x': accuracy_df_without_whitening['Accuracy'].var(), 
                                   'Accuracy_y' : accuracy_df_with_whitening['Accuracy'].var()}


accuracy_df_per_subject.columns= ['', 'Accuracy_wihout_whitening', 'Accuracy_with_whitening']
indexs = []
for i in range(11):
    if i == 4:
        indexs.append("Subjects")
    else:
        indexs.append("")

accuracy_df_per_subject.index = indexs

accuracy_df_per_subject = accuracy_df_per_subject.T


In [4]:
accuracy_df_per_subject

,,,,,Subjects,,,,,,
,Subject_1,Subject_2,Subject_3,Subject_4,Subject_5,Subject_6,Subject_7,Subject_8,Subject_9,Mean,Variance
Accuracy_wihout_whitening,38.75,52.5,50.0,59.52,38.1,52.5,46.25,51.14,46.25,48.334444,46.970103
Accuracy_with_whitening,43.75,53.75,57.5,39.29,50.0,52.5,51.25,55.68,48.75,50.274444,33.129828
